## Imports

In [52]:
import nltk

In [3]:
nltk.download('all')

In [4]:
import numpy as np
import pandas as pd

In [5]:
import re

In [6]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
from sklearn.model_selection import train_test_split

## Dataset prepearing

1 - positive class \
0 - negative class

In [10]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/Restaurant_Reviews.tsv.txt', delimiter="\t", quoting=3)

In [11]:
ps = PorterStemmer()

In [12]:
corpus = []
for i in range(0,1000):
  customer_review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i].lower())
  customer_review = customer_review.split()
  clean_review = [ps.stem(word) for word in customer_review if not word in set(stopwords.words("english"))]
  clean_review = ' '.join(clean_review)
  corpus.append(clean_review)

In [13]:
vectorizer = TfidfVectorizer(max_features=1500, min_df=3, max_df=0.6)

In [14]:
X = vectorizer.fit_transform(corpus).toarray()

In [15]:
y = dataset.iloc[:, 1].values

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Tensorflow Modelling

In [53]:
import tensorflow as tf

In [54]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax'),
    ])

In [56]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [58]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
25/25 [==============================] - 1s 6ms/step - loss: 0.6801 - accuracy: 0.5495
Epoch 2/100
25/25 [==============================] - 0s 6ms/step - loss: 0.4458 - accuracy: 0.8667
Epoch 3/100
25/25 [==============================] - 0s 5ms/step - loss: 0.1927 - accuracy: 0.9352
Epoch 4/100
25/25 [==============================] - 0s 6ms/step - loss: 0.1208 - accuracy: 0.9606
Epoch 5/100
25/25 [==============================] - 0s 6ms/step - loss: 0.0710 - accuracy: 0.9737
Epoch 6/100
25/25 [==============================] - 0s 6ms/step - loss: 0.0348 - accuracy: 0.9890
Epoch 7/100
25/25 [==============================] - 0s 6ms/step - loss: 0.0351 - accuracy: 0.9864
Epoch 8/100
25/25 [==============================] - 0s 6ms/step - loss: 0.0574 - accuracy: 0.9721
Epoch 9/100
25/25 [==============================] - 0s 6ms/step - loss: 0.0322 - accuracy: 0.9904
Epoch 10/100
25/25 [==============================] - 0s 5ms/step - loss: 0.0374 - accuracy: 0.9869
Epoch 11/

In [60]:
loss, accuracy = model.evaluate(X_test, y_test)
loss, accuracy

7/7 [==============================] - 0s 3ms/step - loss: 1.4987 - accuracy: 0.7350


(1.4986717700958252, 0.7350000143051147)

In [61]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               234000    
_________________________________________________________________
dense_1 (Dense)              (None, 500)               250500    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1002      
Total params: 485,502
Trainable params: 485,502
Non-trainable params: 0
_________________________________________________________________


## Check confusion matrix

In [62]:
from sklearn.metrics import confusion_matrix

In [63]:
Ytestpred = model.predict(X_test)

In [64]:
cmknn = confusion_matrix(y_test, y_test_pred)
cmknn

array([[70, 27],
       [24, 79]])

## Check on samples

In [65]:
sample = ["Good batting by England"]

In [66]:
sample = vectorizer.transform(sample).toarray()

In [70]:
sentiment = model.predict(sample)
sentiment

array([[0.2098132, 0.7901868]], dtype=float32)

In [71]:
np.argmax(sentiment)

1

In [72]:
sample2 = ["bad perfomance by India in the match"]
sample2 = vectorizer.transform(sample2).toarray()
sentiment2 = model.predict(sample2)
sentiment2

array([[1.0000000e+00, 3.6861845e-08]], dtype=float32)

In [73]:
np.argmax(sentiment2)

0

## Save model

In [74]:
model.save('text_classifier_model/1')

INFO:tensorflow:Assets written to: text_classifier_model/1/assets
